In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
    

Wczytujemy obraz spring.png za pomocą Pillow

In [ ]:
im= Image.open("spring.png")
X=np.array(im.getdata())
print(X.shape)



Rozkład kolorów w obrazie przed klasteryzacją

In [ ]:
fig = plt.figure(figsize=(10,10))
axis = fig.add_subplot(1, 1, 1, projection="3d")
r, g, b = X[:,0], X[:,1], X[:,2]
axis.scatter(r, g, b, c=X/255.0, marker="o",s=1)
axis.set_xlabel("Red")
axis.set_ylabel("Green")
axis.set_zlabel("Blue")
axis.set_title("Wartości RGB")
plt.show()



Klasteryzacja danych - rysujemy wykres wartości inercji dla poszczególnej ilości klastrów

In [ ]:
import matplotlib.ticker as plticker

inertia_measures = np.empty(0)
arguments = np.arange(1, 17, 1)
for j in range(1, 17):
    clustering = KMeans(n_clusters=j, init='k-means++',
                        max_iter=100, random_state=1)
    clustering.fit(X)
    inertia_measures =np.append(inertia_measures, clustering.inertia_)

inertia_measures=inertia_measures/10**8
fig, ax = plt.subplots()
ax.plot(arguments, inertia_measures, marker='o')
ax.set_xlabel('Liczba klastrów')
ax.set_ylabel('Wartość inercji ^8')
ax.set_title("Wykres wartości inercji")
loc = plticker.MultipleLocator(base=1.0)
ax.xaxis.set_major_locator(loc)
plt.show()

Szukamy pierwszej wartości będącej poniżej limitu 3x10^8

In [ ]:
bestIndex=np.where(inertia_measures<3)[0][0]+1
print(f"Najmniejsza ilośc klastów dla której wartość inercji jest mniejsza niż 3x10^8: {bestIndex}")

Obliczamy współrzędne centroidów dla najlepszego k równego 12

In [ ]:

clustering = KMeans(n_clusters=bestIndex, init='k-means++',
                        max_iter=100, random_state=1).fit(X)

# Przypisujemy środki ciężkości do zmiennych
centroids = clustering.cluster_centers_
#Zobaczmy jak wyglądają środki ciężkości
centroids


Zaokrąglamy środki ciężkości do najbliższej liczby całkowitej

In [ ]:
centroids_rounded = [[round(cord) for cord in rgb_centers] for rgb_centers in centroids]
centroids_rounded = np.array(centroids_rounded)
centroids_rounded 

Grupujemy piksele korzystając z zaokrąglonych punktów ciężkości


In [ ]:

clustering_rounded = KMeans(n_clusters=bestIndex,init=centroids_rounded,
                        max_iter=100,n_init=1).fit(X)
#Pobieramy przypisane piksele
assigments= np.array(clustering_rounded.labels_)



Mapujemy tablicę mapowań do centroidów na tablicę RGB i tworzymy z niej nowy obraz

In [ ]:
pixel_array=[tuple(centroids_rounded[index]) for index in assigments]



image_after_kmeans=Image.new("RGB",(750,482))
image_after_kmeans.putdata(pixel_array)
image_after_kmeans.save("springAfterKMeans.png")
image_after_kmeans.show()




Wypisujemy łączny procent wyjaśnionej przez PCA wariancji

In [ ]:

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
np.sum(pca.explained_variance_ratio_)

In [ ]:
y_kmeans_PCA = clustering.fit_predict(X_pca) 
y_kmeans_PCA

In [ ]:
colors = [tuple(c/255 for c in color) for color in centroids] #skalowanie RGB do przedzialu 0-1
clusters = np.arange(0, len(colors), 1)

Rysujemy wykres punktowy z legendą

In [ ]:
fig, ax = plt.subplots()
cmap=plt.get_cmap('Paired')
for color, i in zip(colors, clusters):
    plt.scatter(X_pca[y_kmeans_PCA == i, 0],
                X_pca[y_kmeans_PCA == i, 1],
                color=color,
                label = i,  # label oznaczający kolor klastra
                cmap=cmap)
plt.title("Wykres punktowy obrazu po PCA")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), # legenda która nie będzie zasłaniać wykresu
          fancybox=True, shadow=True, ncol=5)
plt.show()